In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf

from numpy.random import seed

In [2]:
# definição da seed do python
seed(1)
# e seed do tensorflow
tf.random.set_seed(2)

In [3]:
# definição do ficheiro de dados (treino/validação)
symbol = 'btc'
crypto_ta_dataset_fp = "dataset3/crypto_ta_" + symbol + ".csv"

In [4]:
# Nomes das classses
class_names = ['Very Low', 'Low', 'High', 'Very High']

In [5]:
# ler o ficheiro de dados, indicando o ficheiro e indicamos os nomes das colunas (que não estão no ficheiro)
crypto_ta_dataset = pd.read_csv(crypto_ta_dataset_fp)
# remover as colunas que nao interessam
# crypto_ta_dataset.drop('bbands_up', axis=1, inplace=True)
# crypto_ta_dataset.drop('bbands_mid', axis=1, inplace=True)
# crypto_ta_dataset.drop('bbands_low', axis=1, inplace=True)
# crypto_ta_dataset.drop('macd', axis=1, inplace=True)
# crypto_ta_dataset.drop('macdsignal', axis=1, inplace=True)
# crypto_ta_dataset.drop('macdhist', axis=1, inplace=True)
# crypto_ta_dataset.drop('roc', axis=1, inplace=True)
# crypto_ta_dataset.drop('rocr', axis=1, inplace=True)
# imprime um as primeiras linhas dos dados
print(crypto_ta_dataset.head())

# crypto_ta_dataset['sma_7'] = [x * 100 for x in crypto_ta_dataset['sma_7']]
# crypto_ta_dataset['sma_25'] = [x * 100 for x in crypto_ta_dataset['sma_25']]
# crypto_ta_dataset['sma_99'] = [x * 100 for x in crypto_ta_dataset['sma_99']]
# crypto_ta_dataset['ema_8'] = [x * 100 for x in crypto_ta_dataset['ema_8']]
# crypto_ta_dataset['ema_20'] = [x * 100 for x in crypto_ta_dataset['ema_20']]
# crypto_ta_dataset['bbands_up'] = [x * 100 for x in crypto_ta_dataset['bbands_up']]
# crypto_ta_dataset['bbands_mid'] = [x * 100 for x in crypto_ta_dataset['bbands_mid']]
# crypto_ta_dataset['bbands_low'] = [x * 100 for x in crypto_ta_dataset['bbands_low']]

# colunas que são features e coluna que é a label (a ultima neste caso)
feature_names = crypto_ta_dataset.columns[:-1]
label_name = crypto_ta_dataset.columns[-1]

print("Features: {}".format(feature_names))
print("Label: {}".format(label_name))

      sma_7    sma_25    sma_99   sma_200     ema_8    ema_20    sma_vol  \
0  0.389069  0.180146 -0.065214  0.251872  0.382682  0.219033  -3.532641   
1  0.498987  0.138913 -0.058568  0.255225  0.347571  0.219784  -7.599545   
2  0.231342  0.094671 -0.051021  0.252297  0.117309  0.132675  -8.452679   
3  0.198266  0.129483 -0.046158  0.250075  0.116104  0.130672 -12.107501   
4  0.340448  0.195801 -0.029580  0.260357  0.499679  0.295022 -11.063468   

   bbands_up  bbands_mid  bbands_low  ...        cmo      macd  macdsignal  \
0   0.385451    0.157722   -0.078178  ...  25.485288  2.719071    1.913054   
1   0.279253    0.066455   -0.155002  ...  27.317637  3.153088    2.161061   
2   0.170194    0.062599   -0.049862  ...  17.979856  3.224742    2.373797   
3   0.176183    0.153805    0.130365  ...  19.031101  3.282343    2.555506   
4   0.484177    0.281719    0.069544  ...  33.989614  3.917394    2.827884   

   macdhist       ppo       roc      rocr  linearreg       tsf  trend  
0 

In [6]:
crypto_ta_dataset[label_name] = crypto_ta_dataset[label_name].map(
    {"V_LOW":0,"LOW":1,"HIGH":2,"V_HIGH":3})

print(crypto_ta_dataset)
# criamos duas variáveis, uma para os dados e outra para as labels (vamos precisar depois)
features = crypto_ta_dataset.copy()
labels = features.pop(label_name)
# normalization dataset
# features = tf.keras.utils.normalize(features, axis=-1, order=2)

# imprime um resumo dos valores
# print(features)

          sma_7    sma_25    sma_99   sma_200     ema_8    ema_20    sma_vol  \
0      0.389069  0.180146 -0.065214  0.251872  0.382682  0.219033  -3.532641   
1      0.498987  0.138913 -0.058568  0.255225  0.347571  0.219784  -7.599545   
2      0.231342  0.094671 -0.051021  0.252297  0.117309  0.132675  -8.452679   
3      0.198266  0.129483 -0.046158  0.250075  0.116104  0.130672 -12.107501   
4      0.340448  0.195801 -0.029580  0.260357  0.499679  0.295022 -11.063468   
...         ...       ...       ...       ...       ...       ...        ...   
22700 -0.155741 -0.137875 -0.024924  0.012674 -0.104886 -0.113915  -0.394903   
22701 -0.205752 -0.125204 -0.019734  0.015449  0.050839 -0.046804  -0.509280   
22702 -0.174705 -0.112871 -0.025720  0.016703  0.013208 -0.053574  -0.610728   
22703  0.082261 -0.004702 -0.001615  0.027847  0.322070  0.084389  -0.567466   
22704  0.415341  0.002867 -0.009235  0.029517  0.364738  0.125434  -0.689503   

       bbands_up  bbands_mid  bbands_lo

In [7]:
# sns_plot = sns.pairplot(crypto_ta_dataset,hue=label_name,palette='Set2')
# sns_plot.savefig("output.png")

In [8]:
# definir a estrutra a rede neuronal a utilizar
# Neste caso temos duas camadas escondidas com 10 nós
#     - Ativação do tipo relu (podem usar outras mas podem também escolher manter)
#     - Dense significa que cada camada liga a todas as outras (recomendado)
#     - Na primeira camada escondida indica-se, no parametro input_shape, que temos  entradas
# A camada de saída deve ter o mesmo número de saídas que o número de classes
#     - Por default a camada de saída devolve um logit para cada classe.
#     - Um logit é um numero entre -Inf e +Inf que representa a classificação antes de ser normalizada
#     - Podemos normalizar o resultado depois  para probalidades (usando a função softmax)
#     - Podemos também indicar que esta camada usa logo a softmax mas não é recomendado
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation=tf.nn.relu, input_shape=(20,)),  # input shape required
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(4)
])

In [9]:
# definir as configurações do algoritmo
# - Algoritmo usado para optimização (neste caso o SGD) com a learning rate 
#    - Podem decidir ajustar este parametro mas não é obrigatório
# - Função que será usada para a minimização na procura
#    - Neste caso usamos a SparseCategoricalCrossentropy
#    - SparseCategorical é usada quanda trabalhamos com inteiros como target
#    - from_logits=True indica que a saída da rede são logits
#        - Se não usarmos este parametro temos de usar a softmax na saída da rede
# - As metricas não são usadas para optimização, são usadas para dar output de resultados
#    - Podem-se usar várias métricas ao mesmo tempo, dará vários valores de output
#    - Neste caso estamos a usar a Accuracy (número )

# definir as configurações do algoritmo

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  # Optimizer
    # Loss function to minimize
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    # List of metrics to monitor
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [10]:
# 1- Criar o Dataset
dataset = tf.data.Dataset.from_tensor_slices((features, labels))

# Percentages de teste e validação
TEST_PERC = 0.1
VALID_PERC = 0.1

In [11]:
# 2- Separar por classe (vai permitir garantir que cada conjunto tem X de cada classe)
class0_dataset = dataset.filter(lambda x, y: y == 0)
class1_dataset = dataset.filter(lambda x, y: y == 1)
class2_dataset = dataset.filter(lambda x, y: y == 2)
class3_dataset = dataset.filter(lambda x, y: y == 3)
print()
print(class0_dataset.cardinality())
# quantidade de cada classe e total
DATASIZE = dataset.cardinality().numpy()
c0_size = len(list(class0_dataset))
c1_size = len(list(class1_dataset))
c2_size = len(list(class2_dataset))
c3_size = len(list(class3_dataset))
print(DATASIZE,c0_size,c1_size,c2_size,c3_size)


tf.Tensor(-2, shape=(), dtype=int64)
22705 263 10844 11283 315


In [12]:
# 3- Baralhar
class0_dataset = class0_dataset.shuffle(DATASIZE)
class1_dataset = class1_dataset.shuffle(DATASIZE)
class2_dataset = class2_dataset.shuffle(DATASIZE)
class3_dataset = class3_dataset.shuffle(DATASIZE)

In [13]:
# 4- Partir cada um
# Primeiro retiramos o teste
class0_test = class0_dataset.take(int(c0_size*TEST_PERC))
# variável temporária para separar entre treino e validação
# skip significa que vai ficar com o resto dos dados
rest = class0_dataset.skip(int(c0_size*TEST_PERC))
class0_validation = rest.take(int(c0_size*VALID_PERC))
class0_train = rest.skip(int(c0_size*VALID_PERC))

class1_test = class1_dataset.take(int(c1_size*TEST_PERC))
rest = class1_dataset.skip(int(c1_size*TEST_PERC))
class1_validation = rest.take(int(c1_size*VALID_PERC))
class1_train = rest.skip(int(c1_size*VALID_PERC))

class2_test = class2_dataset.take(int(c2_size*TEST_PERC))
rest = class2_dataset.skip(int(c2_size*TEST_PERC))
class2_validation = rest.take(int(c2_size*VALID_PERC))
class2_train = rest.skip(int(c2_size*VALID_PERC))

class3_test = class3_dataset.take(int(c3_size*TEST_PERC))
rest = class3_dataset.skip(int(c3_size*TEST_PERC))
class3_validation = rest.take(int(c3_size*VALID_PERC))
class3_train = rest.skip(int(c3_size*VALID_PERC))

In [14]:
#5- Juntar tudo novamente
train_dataset = class0_train.concatenate(class1_train).concatenate(class2_train).concatenate(class3_train).shuffle(DATASIZE)
test_dataset = class0_test.concatenate(class1_test).concatenate(class2_test).concatenate(class3_test).shuffle(DATASIZE)
validation_dataset = class0_validation.concatenate(class1_validation).concatenate(class2_validation).concatenate(class3_validation).shuffle(DATASIZE)

#confirmar tamanhos
print('Train dataset size = ', len(list(train_dataset)))
print('Validation dataset size = ', len(list(validation_dataset)))
print('Test dataset size = ', len(list(test_dataset)))

Train dataset size =  18167
Validation dataset size =  2269
Test dataset size =  2269


In [15]:
#6- Usar o dataset diretamente para treinar, validar e testar

# Converter o conjunto de treino de novo para o formato inicial (DataFrame)
feat,lab = map(list,zip(*list(train_dataset.as_numpy_iterator())))
train_features= pd.DataFrame(data=feat,columns=feature_names)
train_labels = pd.DataFrame(data=lab,columns=[label_name])

feat,lab = map(list,zip(*list(validation_dataset.as_numpy_iterator())))
valid_features= pd.DataFrame(data=feat,columns=feature_names)
valid_labels = pd.DataFrame(data=lab,columns=[label_name])

feat,lab = map(list,zip(*list(test_dataset.as_numpy_iterator())))
test_features= pd.DataFrame(data=feat,columns=feature_names)
test_labels = pd.DataFrame(data=lab,columns=[label_name])

In [16]:
# Correr o treino, indica as features as labels e o número de épocas
history = model.fit(train_features, train_labels, epochs=800,validation_data=(valid_features,valid_labels))
print(history)

Epoch 1/800
568/568 [==============================] - 1s 2ms/step - loss: 0.9763 - sparse_categorical_accuracy: 0.4976 - val_loss: 0.8792 - val_sparse_categorical_accuracy: 0.5134
Epoch 2/800
568/568 [==============================] - 1s 1ms/step - loss: 0.8617 - sparse_categorical_accuracy: 0.5135 - val_loss: 0.8291 - val_sparse_categorical_accuracy: 0.5108
Epoch 3/800
568/568 [==============================] - 1s 1ms/step - loss: 0.8371 - sparse_categorical_accuracy: 0.5110 - val_loss: 0.8257 - val_sparse_categorical_accuracy: 0.5209
Epoch 4/800
568/568 [==============================] - 1s 1ms/step - loss: 0.8231 - sparse_categorical_accuracy: 0.5128 - val_loss: 0.8525 - val_sparse_categorical_accuracy: 0.4971
Epoch 5/800
568/568 [==============================] - 1s 1ms/step - loss: 0.8255 - sparse_categorical_accuracy: 0.5140 - val_loss: 0.8302 - val_sparse_categorical_accuracy: 0.5126
Epoch 6/800
568/568 [==============================] - 1s 1ms/step - loss: 0.8148 - sparse_cate

KeyboardInterrupt: 

In [ ]:
# Correr no conjunto teste (com o que foi treinado) e obter o resultado
test = model.evaluate(test_features,test_labels)
print("Teste:\n",test)
loss,acc = test

In [ ]:
# ver um plot dos resultados de treino e teste 
# coloquei também o valor do teste, mas esse é só um ponto no final
# place a text box in upper left in axes coords
# these are matplotlib.patch.Patch properties
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
plt.text(0.3, 0.90, "Acc on test: %.2f" % acc,  fontsize=14, transform=plt.axes().transAxes, 
        verticalalignment='top', bbox=props)
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','validation'])

plt.show()